In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import numpy as np
import tensorflow as tf
import angorapy as apy

env = apy.make_env("LunarLanderContinuous-v2")
distribution = apy.policies.BetaPolicyDistribution(env)

In [4]:
from tensorflow.keras.layers import TimeDistributed
from angorapy.utilities.util import env_extract_dims


def build_my_amazing_model(env, distribution, bs=1, sequence_length=1):
    state_dimensionality, n_actions = env_extract_dims(env)

    inputs = tf.keras.Input(batch_shape=(bs, sequence_length,) + state_dimensionality["proprioception"], name="proprioception")
    masked = tf.keras.layers.Masking(batch_input_shape=(bs, sequence_length,) + (inputs.shape[-1], ))(inputs)

    x = TimeDistributed(tf.keras.layers.Dense(32))(masked)
    x = TimeDistributed(tf.keras.layers.Dense(32))(x)

    x, *_ = tf.keras.layers.LSTM(64,
                       stateful=True,
                       return_sequences=True,
                       return_state=True,
                       batch_size=bs,
                       name="policy_recurrent_layer")(x)

    x_policy = tf.keras.layers.Dense(32)(x)
    x_policy = tf.keras.layers.Dense(32)(x_policy)

    x_value = tf.keras.layers.Dense(32)(x)
    x_value = tf.keras.layers.Dense(32)(x_value)

    out_policy = distribution.build_action_head(n_actions, x_policy.shape[1:], bs)(x_policy)
    out_value = tf.keras.layers.Dense(1)(x_value)

    policy = tf.keras.Model(inputs=inputs, outputs=out_policy, name="my_policy_function")
    value = tf.keras.Model(inputs=inputs, outputs=out_value, name="my_value_function")
    joint = tf.keras.Model(inputs=inputs, outputs=[out_policy, out_value], name="my_joint_networks")

    return policy, value, joint


In [5]:
agent = apy.Agent(build_my_amazing_model, env, horizon=1024, workers=1, distribution=distribution)
agent.drill(n=10, epochs=3, batch_size=32)
agent.save_agent_state()

2023-03-25 12:08:42.262399: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-25 12:08:42.264334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-03-25 12:08:42.297482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.78GiB deviceMemoryBandwidth: 462.00GiB/s
2023-03-25 12:08:42.297680: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-25 12:08:42.304218: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-03-25 12:08:42.304354: I tensorflow/stream_execut

Detected 0 GPU devices.
Using [StateNormalizationTransformer, RewardNormalizationTransformer] for preprocessing.
An MPI Optimizer with 1 ranks has been created; the following ranks optimize: [0]


Drill started using 1 processes for 1 workers of which 1 are optimizers. Worker distribution: [1].
IDs over Workers: [[0]]
IDs over Optimizers: [[0]]

The policy is recurrent and the batch size is interpreted as the number of transitions per policy update. Given the batch size of 32 this results in: 
	2 chunks per update and 32 updates per epoch
	Batch tilings of (1, 2) per process and (1, 2) in total.




2023-03-25 12:08:43.576110: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Gathering cycle 0...

2023-03-25 12:08:44.558125: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-25 12:08:44.575742: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz


...

Cycle     8/10: r:   -90.82; len:   123.29; n:   7; loss: [ -0.44|    0.01| -0.39]; eps:    65; lr: 1.00e-03; upd:    768; f:    8.192k; w: 0.06; times: [9.1|0.0|6.2] [59|0|41]; took 14.81s [0.5mins left]; mem: 1.27/33|0.0/0.0;
Optimizing...

Optimizing...:   3%|▎         | 96/3072 [00:06<03:09, 15.74it/s]


Cycle     9/10: r:   -68.38; len:   116.38; n:   8; loss: [ -0.23|    0.01| -0.22]; eps:    72; lr: 1.00e-03; upd:    864; f:    9.216k; w: 0.04; times: [8.4|0.0|6.3] [57|0|43]; took 14.96s [0.3mins left]; mem: 1.27/33|0.0/0.0;
Optimizing...

Optimizing...:   3%|▎         | 96/3072 [00:06<03:09, 15.73it/s]


Finalizing...Drill finished after 153.46serialization.


In [6]:
evaluation_results = agent.evaluate(10, act_confidently=True)[0]
print(np.mean(evaluation_results.episode_rewards))

-162.23110202608896
